In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.5/558.5 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.4/359.4 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import tensorflow as tf
import tensorflow_federated as tff

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/LAB3-20221109T160610Z-001/LCL-FullData/Cluster14data.csv')

In [ ]:
# Data structing for testing the TFF Functions
data['DateTime'] = pd.to_datetime(data['DateTime'])

# Filter the DataFrame to include readings from Jan 01, 2013, to Feb 28, 2013
start_date1 = pd.to_datetime('2013-01-01')
end_date1 = pd.to_datetime('2013-02-28')
filtered_data1 = data[(data['DateTime'] >= start_date1) & (data['DateTime'] <= end_date1)]

In [ ]:
lclid_list1 = filtered_data1['LCLid'].unique()
selected_lclids1 = lclid_list1[:20]
f_data1 = filtered_data1[filtered_data1['LCLid'].isin(selected_lclids1)]

In [ ]:
import numpy as np


In [ ]:
datan1 = f_data1.copy()
datan1['KWH/hh'] = datan1['KWH/hh'].astype(np.float32)

In [ ]:
f_data1 = f_data1.drop('cluster', axis=1)
f_data1 = f_data1.drop('stdorToU', axis=1)
datan1 = datan1.drop('cluster', axis=1)
datan1 = datan1.drop('stdorToU', axis=1)

In [ ]:
datan1.reset_index(drop=True, inplace=True)
datan1['DateTime'] = pd.to_datetime(datan1.DateTime).dt.tz_localize(None)
for i in range(len(datan1)):
  datan1['DateTime'][i]=datan1['DateTime'][i].timestamp()

<ipython-input-11-723fbda6ea1d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datan1['DateTime'][i]=datan1['DateTime'][i].timestamp()


In [ ]:
datan1['DateTime'] = datan1['DateTime'].astype(np.float32)
# Sort the data by 'LCLid' and 'DateTime'
datan1.sort_values(['LCLid', 'DateTime'], inplace=True)

In [ ]:
import collections

In [ ]:
# SLiding Window Function
def create_client_dataset_for_LCLid(client_data, window_size, step_size):
    client_windows = []
    client_targets = []
    num_readings = len(client_data)

    # Iterate over the readings using the sliding window
    for i in range(0, num_readings - window_size, step_size):
        window_start = i
        window_end = i + window_size - 1
        prediction_index = window_end + step_size

        # Extract the window and the prediction target
        window = client_data.iloc[window_start:window_end + 1]['KWH/hh'].values
        target = client_data.iloc[prediction_index]['KWH/hh']

        client_windows.append(window)
        client_targets.append(target)

    # Create an ordered dictionary with 'x' and 'y' keys
    ordered_dict = collections.OrderedDict()
    ordered_dict['x'] = tf.stack(client_windows)
    ordered_dict['y'] = tf.expand_dims(client_targets, axis=-1)


    return ordered_dict

In [ ]:
window_size = 336
step_size = 1

# Filter the dataframe for the specific LCLid
example_LCLid = datan1['LCLid'].unique()[3]
clientyy_data = datan1[datan1['LCLid'] == example_LCLid]

# Create the client dataset for the specific LCLid
example_client_dataset = create_client_dataset_for_LCLid(clientyy_data, window_size, step_size)

In [ ]:
NUM_EPOCHS = 5
BATCH_SIZE = 20
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10

def preprocess_client_dataset(dataset):
    def batch_format_fn(element):
        return collections.OrderedDict(
            x=tf.reshape(element['x'], [-1, 336]),
            y=tf.reshape(element['y'], [-1, 1]))
    return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
        BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)


preprocessed_example_client_dataset = preprocess_client_dataset(tf.data.Dataset.from_tensor_slices(example_client_dataset))


sample_batch = tf.nest.map_structure(lambda x: x.numpy(),
                                     next(iter(preprocessed_example_client_dataset)))

sample_batch

OrderedDict([('x',
              array([[0.034, 0.068, 0.058, ..., 0.023, 0.025, 0.065],
                     [0.148, 0.439, 0.114, ..., 0.013, 0.04 , 0.066],
                     [0.069, 0.021, 0.048, ..., 0.051, 0.035, 0.019],
                     ...,
                     [0.149, 0.058, 0.09 , ..., 0.139, 0.218, 0.146],
                     [0.034, 0.013, 0.043, ..., 0.028, 0.055, 0.043],
                     [0.013, 0.013, 0.053, ..., 0.104, 0.081, 0.018]], dtype=float32)),
             ('y',
              array([[0.026],
                     [0.096],
                     [0.051],
                     [0.03 ],
                     [0.018],
                     [0.017],
                     [0.013],
                     [0.043],
                     [0.081],
                     [0.012],
                     [0.035],
                     [0.013],
                     [0.048],
                     [0.095],
                     [0.124],
                     [0.013],
                  

In [ ]:
NUM_CLIENTS = 10  # Replace with desired number of clients
sample_clients1 = datan1['LCLid'].unique()[0:NUM_CLIENTS]
sample_clients_list1 = sample_clients1.tolist()

In [ ]:
# Iterate over unique LCLids in the dataframe
client_datasets1 = {}
for LCLid in sample_clients_list1:
    # Filter the dataframe for the current LCLid
    client_data = datan1[datan1['LCLid'] == LCLid]

    clientxx_dataset = create_client_dataset_for_LCLid(client_data, window_size, step_size)

    # Create the client dataset for the current LCLid
    preprocessed_client_dataset = preprocess_client_dataset(tf.data.Dataset.from_tensor_slices(clientxx_dataset))

    # Extract a sample batch from the preprocessed dataset
    sam_batch = tf.nest.map_structure(lambda x: x.numpy(), next(iter(preprocessed_client_dataset)))

    # Store the preprocessed dataset in the dictionary with LCLid as the key
    client_datasets1[LCLid] = preprocessed_client_dataset

In [ ]:
def make_federated_data(client_datasets, sample_clients_list):
    return [
        client_datasets[x] for x in sample_clients_list
    ]

In [ ]:
federated_train_data0 = make_federated_data(client_datasets1, sample_clients1)

In [ ]:
def create_keras_model():
  model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(336,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
  return model

In [ ]:
def model_fn():
    keras_model = create_keras_model()
    loss = tf.keras.losses.MeanSquaredError()
    tff_model = tff.learning.models.from_keras_model(
        keras_model,
        input_spec=preprocessed_example_client_dataset.element_spec,
        loss=loss,
        metrics=[tf.keras.metrics.MeanAbsoluteError()]
    )
    return tff_model

In [ ]:
# Define Client Update Functionality
@tf.function
def client_update(model, dataset, server_weights, client_optimizer):
  """Performs training (using the server model weights) on the client's dataset."""
  # Initialize the client model with the current server weights.
  client_weights = model.trainable_variables
  # Assign the server weights to the client model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)

  # Use the client_optimizer to update the local model.
  for batch in dataset:
    with tf.GradientTape() as tape:
      # Compute a forward pass on the batch of data
      outputs = model.forward_pass(batch)

    # Compute the corresponding gradient
    grads = tape.gradient(outputs.loss, client_weights)
    grads_and_vars = zip(grads, client_weights)

    # Apply the gradient using a client optimizer.
    client_optimizer.apply_gradients(grads_and_vars)

  return client_weights

In [ ]:
# Define Gloabal Server Update Functionality
@tf.function
def server_update(model, mean_client_weights):
  """Updates the server model weights as the average of the client model weights."""
  model_weights = model.trainable_variables
  # Assign the mean client weights to the server model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        model_weights, mean_client_weights)
  return model_weights

In [ ]:
# Initialize Function at Server
@tff.tf_computation
def server_init():
  model = model_fn()
  return model.trainable_variables

In [ ]:
@tff.federated_computation
def initialize_fn():
  return tff.federated_value(server_init(), tff.SERVER)

In [ ]:
whimsy_model = model_fn()
tf_dataset_type = tff.SequenceType(whimsy_model.input_spec)

In [ ]:
tf_dataset_type

SequenceType(StructType([('x', TensorType(tf.float32, [None, 336])), ('y', TensorType(tf.float32, [None, 1]))]) as OrderedDict)

In [ ]:
str(tf_dataset_type)

'<x=float32[?,336],y=float32[?,1]>*'

In [ ]:
model_weights_type = server_init.type_signature.result

In [ ]:
str(model_weights_type)

'<float32[336,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>'

In [ ]:
@tff.tf_computation(tf_dataset_type, model_weights_type)
def client_update_fn(tf_dataset, server_weights):
  model = model_fn()
  client_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
  return client_update(model, tf_dataset, server_weights, client_optimizer)

In [ ]:
@tff.tf_computation(model_weights_type)
def server_update_fn(mean_client_weights):
  model = model_fn()
  return server_update(model, mean_client_weights)

In [ ]:
federated_server_type = tff.FederatedType(model_weights_type, tff.SERVER)
federated_dataset_type = tff.FederatedType(tf_dataset_type, tff.CLIENTS)

In [ ]:
@tff.federated_computation(federated_server_type, federated_dataset_type)
def next_fn(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = tff.federated_broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = tff.federated_map(
      client_update_fn, (federated_dataset, server_weights_at_client))

  # The server averages these updates.
  mean_client_weights = tff.federated_mean(client_weights)

  # The server updates its model.
  server_weights = tff.federated_map(server_update_fn, mean_client_weights)

  return server_weights, mean_client_weights, client_weights, server_weights_at_client

In [ ]:
federated_algorithm = tff.templates.IterativeProcess(
    initialize_fn=initialize_fn,
    next_fn=next_fn
)

In [ ]:
str(federated_algorithm.initialize.type_signature)

'( -> <float32[336,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>@SERVER)'

In [ ]:
str(federated_algorithm.next.type_signature)

'(<server_weights=<float32[336,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>@SERVER,federated_dataset={<x=float32[?,336],y=float32[?,1]>*}@CLIENTS> -> <<float32[336,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>@SERVER,<float32[336,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>@SERVER,{<float32[336,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>}@CLIENTS,<float32[336,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>@CLIENTS>)'

In [ ]:
server_state = federated_algorithm.initialize()


In [ ]:
server_state

[array([[ 0.02173326, -0.11624512, -0.07785487, ...,  0.00381591,
          0.03158902,  0.08821423],
        [ 0.12152595,  0.06697931,  0.04509386, ...,  0.04036345,
         -0.06315383,  0.09464776],
        [ 0.0219169 ,  0.10069706, -0.11798855, ...,  0.04890331,
         -0.11993705, -0.06072101],
        ...,
        [ 0.04573313, -0.06253831,  0.09135447, ...,  0.11636645,
          0.0319015 , -0.08920309],
        [ 0.05695326,  0.06568489,  0.06095756, ...,  0.00538539,
         -0.03768393, -0.02144573],
        [-0.06918408, -0.09427081,  0.12075906, ...,  0.05197345,
          0.0636775 ,  0.01131945]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 array([[-0.08826214, -0.17727208,  0.19703037, ..., 

In [ ]:
result = federated_algorithm.next(server_state, federated_train_data0)


In [ ]:
result

([array([[ 0.02365782, -0.11741928, -0.0800062 , ...,  0.01085825,
           0.03006899,  0.08584992],
         [ 0.12330018,  0.06573105,  0.04327314, ...,  0.04648448,
          -0.06341912,  0.09220649],
         [ 0.02312181,  0.09938306, -0.11990335, ...,  0.0540356 ,
          -0.11990057, -0.06186141],
         ...,
         [ 0.04754342, -0.06352425,  0.09087066, ...,  0.12705284,
           0.03344471, -0.09232201],
         [ 0.058374  ,  0.06426404,  0.05989562, ...,  0.01917936,
          -0.03568754, -0.02563901],
         [-0.06911206, -0.0965618 ,  0.11881167, ...,  0.07497874,
           0.06621419,  0.00523745]], dtype=float32),
  array([ 1.5513532e-02, -2.3433528e-04, -6.2115863e-03,  5.5939336e-03,
         -7.7581359e-03, -7.1792169e-03, -4.1775121e-03, -7.8888992e-03,
         -1.7731792e-03,  2.3291472e-03,  3.4705792e-03, -6.2018062e-04,
         -7.6063639e-03,  2.7979715e-03, -1.0680328e-02,  3.1700316e-03,
         -3.6836942e-03, -1.6563462e-02,  1.2248645e-